In [ ]:
import os
import rasterio
import numpy as np
import tacoreader
import pandas as pd

folder_path = '../src'
os.chdir(folder_path)
from dataset.download_data import download_cloudsen12
from utils.data_loader import create_dataloaders

In [2]:
# Baixar os dados (caso ainda não estejam baixados)
parts = download_cloudsen12(local_dir="../data/dados", type = "L1C")
print("Arquivos baixados:", parts)

Arquivos baixados: ['..\\data\\dados\\cloudsen12-l1c.0000.part.taco', '..\\data\\dados\\cloudsen12-l1c.0004.part.taco']


In [3]:
# Carregar o dataset
ds = tacoreader.load(parts)
print("Número total de amostras:", len(ds))

Número total de amostras: 16135


In [4]:
print(ds["tortilla:data_split"].value_counts())

tortilla:data_split
train         13248
test           1715
validation     1172
Name: count, dtype: int64


- A contagem global (47360 train / 1715 test / 1172 validation) é o total de todas as amostras do CloudSEN12+

In [5]:
ds = tacoreader.load(parts)
sample = ds.read(10)
sample

,internal:subfile,tortilla:id,tortilla:file_format,tortilla:data_split,tortilla:offset,tortilla:length,stac:crs,stac:geotransform,stac:raster_shape,stac:time_start,stac:time_end,stac:centroid
0,"/vsisubfile/7235135_1359647,..\data\dados\clou...",s2l1c,GTiff,None,7235135,1359647,EPSG:32719,"[502160.0, 10.0, 0.0, 7567380.0, 0.0, -10.0]","[512, 512]",1.549637e+09,1.549637e+09,POINT (-68.954266 -22.021252)
1,"/vsisubfile/8594782_7153,..\data\dados\cloudse...",target,GTiff,None,8594782,7153,EPSG:32719,"[502160.0, 10.0, 0.0, 7567380.0, 0.0, -10.0]","[512, 512]",1.549637e+09,1.549637e+09,POINT (-68.954266 -22.021252)


In [6]:
img_path = sample.read(0)
with rasterio.open(img_path) as src:
    # Exibe os metadados da imagem (incluindo o dtype e o número de bandas)
    print("Metadata:", src.meta)
    
    # Itera sobre cada banda para obter o tipo de dado e o range
    for band in range(1, src.count + 1):
        band_data = src.read(band)
        print(f"Banda {band}: dtype={band_data.dtype}, mínimo={np.min(band_data)}, máximo={np.max(band_data)}")


Metadata: {'driver': 'GTiff', 'dtype': 'uint16', 'nodata': 65535.0, 'width': 512, 'height': 512, 'count': 13, 'crs': CRS.from_epsg(32719), 'transform': Affine(10.0, 0.0, 502160.0,
       0.0, -10.0, 7567380.0)}
Banda 1: dtype=uint16, mínimo=0, máximo=3968
Banda 2: dtype=uint16, mínimo=0, máximo=3932
Banda 3: dtype=uint16, mínimo=0, máximo=3876
Banda 4: dtype=uint16, mínimo=0, máximo=4476
Banda 5: dtype=uint16, mínimo=0, máximo=4736
Banda 6: dtype=uint16, mínimo=0, máximo=4972
Banda 7: dtype=uint16, mínimo=0, máximo=4960
Banda 8: dtype=uint16, mínimo=0, máximo=4652
Banda 9: dtype=uint16, mínimo=0, máximo=4720
Banda 10: dtype=uint16, mínimo=0, máximo=2856
Banda 11: dtype=uint16, mínimo=0, máximo=492
Banda 12: dtype=uint16, mínimo=0, máximo=6004
Banda 13: dtype=uint16, mínimo=0, máximo=4000


In [7]:
df = pd.DataFrame(ds)
print("Colunas do DataFrame:", df.columns)

Colunas do DataFrame: Index(['internal:subfile', 'tortilla:id', 'tortilla:file_format',
       'tortilla:data_split', 'tortilla:offset', 'tortilla:length', 'stac:crs',
       'stac:geotransform', 'stac:raster_shape', 'stac:time_start',
       'stac:time_end', 'stac:centroid', 'rai:ele', 'rai:cisi', 'rai:gdp',
       'rai:hdi', 'rai:gmi', 'rai:pop', 'rai:admin0', 'rai:admin1',
       'rai:admin2', 'roi_id', 'old_roi_id', 'equi_id', 'equi_zone',
       'label_type', 's2_id', 'real_proj_shape', 's2_mean_solar_azimuth_angle',
       's2_mean_solar_zenith_angle', 'thick_percentage', 'thin_percentage',
       'cloud_shadow_percentage', 'clear_percentage'],
      dtype='object')


**Treinar primeiro em p509** (que possui mais amostras e, portanto, fornece uma diversidade ampla para a rede aprender as classes de nuvem).  
**Depois refinar (fine-tuning) com p2000**, pois os patches maiores ajudam a capturar melhor as correlações espaciais entre nuvens e sombras, o que é benéfico especialmente para a detecção de sombras.

Em resumo:
  
**Treino principal em p509**: aproveita-se a grande quantidade de patches com rótulos “high” (além de “scribble” e “nolabel”, se desejar) para uma base sólida.  
**Fine-tuning em p2000**: complementa o aprendizado, fornecendo amostras bem maiores, nas quais o modelo pode aprender melhor a relação geométrica entre nuvens e suas sombras.

Em termos de implementação, você pode:
- Primeiramente carregar apenas p509 (“real_proj_shape=509”) nos seus DataLoaders, treinar até convergência.  
- Em seguida, iniciar um treinamento (ou fine-tuning) carregando apenas p2000 (“real_proj_shape=2000”). Nesse ponto, ou você retoma o estado do modelo anterior (carregando o checkpoint da rede treinada em p509) e faz alguns epochs extras, ou congela camadas, ou usa outra estratégia de transferência, conforme a arquitetura e práticas de fine-tuning que preferir.

No artigo, os **três tipos de label_type** aparecem descritos assim:

**high**  
   - Patches com anotações densas (cada pixel é classificado em clear, thick cloud, thin cloud ou cloud shadow).  
   - Divididos em train, val, e test.  
   - Ideais para **treinamento supervisionado** tradicional, pois cada pixel tem um rótulo confiável.  

**scribble**  
   - Patches onde apenas um pequeno conjunto de pixels (menos de 5%) foi anotado manualmente, em formato de “rabiscos” (daí o nome scribble).  
   - Também divididos em train, val e test.  
   - Úteis para **validação** e para experimentos de semi-supervisão, pois oferecem alguma informação de rótulo em poucos pixels, ajudando a corrigir erros em regiões críticas (bordas de nuvem, por exemplo).

**nolabel**  
   - Patches **sem** anotações humanas.  
   - Disponíveis apenas na pasta *train*.  
   - O artigo menciona que cada um desses patches recebe uma máscara de nuvens inferida automaticamente pelo modelo UnetMobV2, podendo servir como base para pré-treinamento (pseudo-rótulos) ou para estratégias de aprendizado semi-supervisionado.  

Em suma, **“high”** é o principal conjunto supervisionado, **“scribble”** serve como conjunto adicional de validação/treinamento parcial, e **“nolabel”** oferece um grande volume de dados sem anotação manual, mas com máscaras geradas automaticamente para quem quiser explorar técnicas de pseudo-rótulo, auto-treinamento ou aprendizado semi-supervisionado.

In [ ]:
train_loader, val_loader, test_loader = create_dataloaders(
    parts,
    real_proj_shape=509,  # ou 2000 
    label_type="high",    # "scribble" ou "nolabel"
    batch_size=8,
    num_workers=2
)
# Verificar quantos batches e a forma dos tensores
if train_loader is not None:
    for idx, (imgs, masks) in enumerate(train_loader):
        print(f"Lote {idx}: imagens={imgs.shape}, máscaras={masks.shape}")
        break

Train samples: 8490
Val samples:   535
Test samples:  975
